In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json
import six

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras import regularizers, initializers
from keras.layers.advanced_activations import LeakyReLU, ReLU, Softmax
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras import backend as K
from keras.regularizers import l2
from keras.layers.merge import add

Using TensorFlow backend.


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True,augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
12937,resized/12939.jpg,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0
3086,resized/3087.jpg,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
5553,resized/5554.jpg,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0
6211,resized/6212.jpg,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0
8552,resized/8553.jpg,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)


def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f


def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input

    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.common.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.common.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

In [0]:
backbone = ResnetBuilder.build_resnet_18((3, 224, 224), 100)
neck = backbone.output

def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
#Triangle cyclic learning rate policy

from keras.callbacks import *

class CyclicLR(Callback):
    

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
clr = CyclicLR(base_lr=0.001, max_lr=0.1,
                        step_size=782.)
#SDG 
sgd = SGD(lr=0.001, momentum = 1e-2)

In [0]:
 losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "categorical_crossentropy",
 	"weight_output": "categorical_crossentropy",}
 loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
from keras import callbacks

#lrsched = keras.callbacks.LearningRateScheduler(5, verbose=0)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.001)
model.compile(
    optimizer=sgd,
    loss="categorical_crossentropy", 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
train_gen = PersonDataGenerator(train_df, batch_size=512, augmentation=ImageDataGenerator(
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        vertical_flip=True,
        featurewise_center=True,
        featurewise_std_normalization=True,zca_whitening=True))
valid_gen = PersonDataGenerator(val_df, batch_size=512, shuffle=False)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:341: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
from keras.callbacks import ModelCheckpoint
filepath = '/content/gdrive/My Drive/21trial{epoch}.hdf5'
ckpt =ModelCheckpoint(filepath, verbose=1, monitor='val_loss', save_best_only=True,save_weights_only=False, mode='auto',period=1)
callback = [ckpt,reduce_lr,clr]

In [0]:

model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    callbacks=callback
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
22/22 [==============================] - 39s 2s/step - loss: 10.4081 - gender_output_loss: 0.6927 - image_quality_output_loss: 1.0961 - age_output_loss: 1.6040 - weight_output_loss: 1.3930 - bag_output_loss: 1.0991 - footwear_output_loss: 1.0968 - pose_output_loss: 1.0980 - emotion_output_loss: 1.3674 - gender_output_acc: 0.5402 - image_quality_output_acc: 0.3849 - age_output_acc: 0.3064 - weight_output_acc: 0.1353 - bag_output_acc: 0.2964 - footwear_output_acc: 0.3822 - pose_output_acc: 0.3032 - emotion_output_acc: 0.7053 - val_loss: 10.3528 - val_gender_output_loss: 0.6923 - val_image_quality_output_loss: 1.0915 - val_age_output_loss: 1.5997 - val_weight_output_loss: 1.3771 - val_bag_output_loss: 1.0926 - val_footwear_output_loss: 1.0945 - val_pose_output_loss: 1.0922 - val_emotion_output_loss: 1.3518 - val_gender_output_acc: 0.5605 - val_image_quality_output_acc: 0.5293 - val_a

In [0]:
train_gen = PersonDataGenerator(train_df, batch_size=128, augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        featurewise_center=True,
        featurewise_std_normalization=True))
valid_gen = PersonDataGenerator(val_df, batch_size=128, shuffle=False)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,                                                
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    callbacks=callback
)

Epoch 1/50
89/90 [============================>.] - ETA: 0s - loss: 8.6973 - gender_output_loss: 0.6828 - image_quality_output_loss: 0.9689 - age_output_loss: 1.4218 - weight_output_loss: 0.9807 - bag_output_loss: 0.9146 - footwear_output_loss: 0.9666 - pose_output_loss: 0.9250 - emotion_output_loss: 0.9015 - gender_output_acc: 0.5629 - image_quality_output_acc: 0.5573 - age_output_acc: 0.3977 - weight_output_acc: 0.6352 - bag_output_acc: 0.5642 - footwear_output_acc: 0.5381 - pose_output_acc: 0.6185 - emotion_output_acc: 0.7130

90/90 [==============================] - 33s 364ms/step - loss: 8.6991 - gender_output_loss: 0.6828 - image_quality_output_loss: 0.9691 - age_output_loss: 1.4228 - weight_output_loss: 0.9807 - bag_output_loss: 0.9145 - footwear_output_loss: 0.9662 - pose_output_loss: 0.9251 - emotion_output_loss: 0.9026 - gender_output_acc: 0.5626 - image_quality_output_acc: 0.5573 - age_output_acc: 0.3973 - weight_output_acc: 0.6352 - bag_output_acc: 0.5637 - footwear_output_

In [0]:
model.load_weights('/content/gdrive/My Drive/21trial37.hdf5')

In [0]:
results=model.evaluate_generator(valid_gen, verbose=1)
dict(zip(model.metrics_names,results))


15/15 [==============================] - 2s 150ms/step


{'age_output_acc': 0.40520833333333334,
 'age_output_loss': 1.4085518916447957,
 'bag_output_acc': 0.5791666666666667,
 'bag_output_loss': 0.8869395971298217,
 'emotion_output_acc': 0.7026041666666667,
 'emotion_output_loss': 0.9138497749964396,
 'footwear_output_acc': 0.6171875,
 'footwear_output_loss': 0.8998642683029174,
 'gender_output_acc': 0.6734375,
 'gender_output_loss': 0.5864527265230814,
 'image_quality_output_acc': 0.5291666666666667,
 'image_quality_output_loss': 0.9892781058947245,
 'loss': 8.450695737202961,
 'pose_output_acc': 0.6130208333333333,
 'pose_output_loss': 0.9171522259712219,
 'weight_output_acc': 0.6369791666666667,
 'weight_output_loss': 0.9700624783833821}

In [0]:
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        featurewise_center=True,
        featurewise_std_normalization=True))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    callbacks=callback
)

Epoch 1/50
360/360 [==============================] - 41s 114ms/step - loss: 8.4114 - gender_output_loss: 0.6199 - image_quality_output_loss: 0.9672 - age_output_loss: 1.4098 - weight_output_loss: 0.9769 - bag_output_loss: 0.8978 - footwear_output_loss: 0.8423 - pose_output_loss: 0.9213 - emotion_output_loss: 0.9004 - gender_output_acc: 0.6577 - image_quality_output_acc: 0.5575 - age_output_acc: 0.3978 - weight_output_acc: 0.6352 - bag_output_acc: 0.5791 - footwear_output_acc: 0.6263 - pose_output_acc: 0.6184 - emotion_output_acc: 0.7123 - val_loss: 8.4650 - val_gender_output_loss: 0.6010 - val_image_quality_output_loss: 0.9903 - val_age_output_loss: 1.4063 - val_weight_output_loss: 0.9698 - val_bag_output_loss: 0.8841 - val_footwear_output_loss: 0.9097 - val_pose_output_loss: 0.9232 - val_emotion_output_loss: 0.9076 - val_gender_output_acc: 0.6668 - val_image_quality_output_acc: 0.5307 - val_age_output_acc: 0.4068 - val_weight_output_acc: 0.6371 - val_bag_output_acc: 0.5771 - val_foot

In [0]:
#model.load_weights('/content/gdrive/My Drive/21trial6.hdf5')

In [0]:
results=model.evaluate_generator(valid_gen, verbose=1)
dict(zip(model.metrics_names,results))

31/31 [==============================] - 3s 84ms/step


{'age_output_acc': 0.4122983870967742,
 'age_output_loss': 1.3948809792918544,
 'bag_output_acc': 0.6008064516129032,
 'bag_output_loss': 0.8639771323050222,
 'emotion_output_acc': 0.7051411290322581,
 'emotion_output_loss': 0.8999442035152067,
 'footwear_output_acc': 0.623991935483871,
 'footwear_output_loss': 1.1796005887369956,
 'gender_output_acc': 0.7706653225806451,
 'gender_output_loss': 0.8094604236464347,
 'image_quality_output_acc': 0.5352822580645161,
 'image_quality_output_loss': 0.9692999143754283,
 'loss': 8.643104137912873,
 'pose_output_acc': 0.6517137096774194,
 'pose_output_loss': 0.8266492543681976,
 'weight_output_acc': 0.639616935483871,
 'weight_output_loss': 0.9677336196745595}

In [0]:
def evaluate_model(model):
    results = model.evaluate_generator(valid_gen, verbose=1)
    accuracies = {}
    losses = {}
    for k, v in zip(model.metrics_names, results):
        if k.endswith('acc'):
            accuracies[k] = round(v * 100, 4) 
        else:
            losses[k] = v
    return accuracies

In [0]:
evaluate_model(model)

31/31 [==============================] - 3s 83ms/step


{'age_output_acc': 41.2298,
 'bag_output_acc': 60.0806,
 'emotion_output_acc': 70.5141,
 'footwear_output_acc': 62.3992,
 'gender_output_acc': 77.0665,
 'image_quality_output_acc': 53.5282,
 'pose_output_acc': 65.1714,
 'weight_output_acc': 63.9617}

In [0]:
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        featurewise_center=True,
        featurewise_std_normalization=True))
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=100,
    verbose=1, 
    callbacks=callback
    )                                              

Epoch 1/100
360/360 [==============================] - 42s 116ms/step - loss: 6.8324 - gender_output_loss: 0.0897 - image_quality_output_loss: 0.8803 - age_output_loss: 1.3257 - weight_output_loss: 0.9325 - bag_output_loss: 0.8326 - footwear_output_loss: 0.4319 - pose_output_loss: 0.7557 - emotion_output_loss: 0.8542 - gender_output_acc: 0.9688 - image_quality_output_acc: 0.6005 - age_output_acc: 0.4233 - weight_output_acc: 0.6451 - bag_output_acc: 0.6350 - footwear_output_acc: 0.8135 - pose_output_acc: 0.6818 - emotion_output_acc: 0.7151 - val_loss: 8.9991 - val_gender_output_loss: 0.9070 - val_image_quality_output_loss: 1.0734 - val_age_output_loss: 1.4084 - val_weight_output_loss: 0.9736 - val_bag_output_loss: 0.8835 - val_footwear_output_loss: 1.2877 - val_pose_output_loss: 0.8352 - val_emotion_output_loss: 0.9020 - val_gender_output_acc: 0.7738 - val_image_quality_output_acc: 0.5089 - val_age_output_acc: 0.4048 - val_weight_output_acc: 0.6354 - val_bag_output_acc: 0.6042 - val_foo

In [0]:
model.load_weights('/content/gdrive/My Drive/21trial13.hdf5')

In [0]:
results=model.evaluate_generator(valid_gen, verbose=1)            
dict(zip(model.metrics_names,results))

63/63 [==============================] - 3s 48ms/step


{'age_output_acc': 0.40922619047619047,
 'age_output_loss': 1.8777607092781672,
 'bag_output_acc': 0.6026785714285714,
 'bag_output_loss': 0.8616279269021655,
 'emotion_output_acc': 0.7058531746031746,
 'emotion_output_loss': 0.8822168792997088,
 'footwear_output_acc': 0.6254960317460317,
 'footwear_output_loss': 1.2741732597351074,
 'gender_output_acc': 0.7619047619047619,
 'gender_output_loss': 1.2239927204828414,
 'image_quality_output_acc': 0.5376984126984127,
 'image_quality_output_loss': 2.2497570457912626,
 'loss': 11.670529108198862,
 'pose_output_acc': 0.6919642857142857,
 'pose_output_loss': 1.337195474950094,
 'weight_output_acc': 0.6329365079365079,
 'weight_output_loss': 1.2229907796496438}

In [0]:
evaluate_model(model)

63/63 [==============================] - 3s 47ms/step


{'age_output_acc': 40.9226,
 'bag_output_acc': 60.2679,
 'emotion_output_acc': 70.5853,
 'footwear_output_acc': 62.5496,
 'gender_output_acc': 76.1905,
 'image_quality_output_acc': 53.7698,
 'pose_output_acc': 69.1964,
 'weight_output_acc': 63.2937}

In [0]:
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        ))
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False)

In [0]:
model.load_weights('/content/gdrive/My Drive/21trial6.hdf5')

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8, 
    epochs=100,
    verbose=1,
    callbacks=callback
)

Epoch 1/100
360/360 [==============================] - 41s 115ms/step - loss: 8.0781 - gender_output_loss: 0.4983 - image_quality_output_loss: 0.9634 - age_output_loss: 1.3921 - weight_output_loss: 0.9754 - bag_output_loss: 0.8672 - footwear_output_loss: 0.7204 - pose_output_loss: 0.9169 - emotion_output_loss: 0.8945 - gender_output_acc: 0.7528 - image_quality_output_acc: 0.5577 - age_output_acc: 0.4014 - weight_output_acc: 0.6354 - bag_output_acc: 0.6165 - footwear_output_acc: 0.6939 - pose_output_acc: 0.6188 - emotion_output_acc: 0.7123 - val_loss: 8.5736 - val_gender_output_loss: 0.6471 - val_image_quality_output_loss: 0.9911 - val_age_output_loss: 1.4147 - val_weight_output_loss: 0.9753 - val_bag_output_loss: 0.8991 - val_footwear_output_loss: 0.9620 - val_pose_output_loss: 0.9303 - val_emotion_output_loss: 0.9060 - val_gender_output_acc: 0.6508 - val_image_quality_output_acc: 0.5303 - val_age_output_acc: 0.4077 - val_weight_output_acc: 0.6359 - val_bag_output_acc: 0.5665 - val_foo

In [0]:
 results=model.evaluate_generator(valid_gen, verbose=1)            
dict(zip(model.metrics_names,results))
evaluate_model(model)

63/63 [==============================] - 3s 47ms/step


{'age_output_acc': 0.40327380952380953,
 'age_output_loss': 1.5832837328078255,
 'bag_output_acc': 0.589781746031746,
 'bag_output_loss': 0.8689437518044124,
 'emotion_output_acc': 0.7068452380952381,
 'emotion_output_loss': 0.8838645380640787,
 'footwear_output_acc': 0.6259920634920635,
 'footwear_output_loss': 1.3977487806289914,
 'gender_output_acc': 0.7941468253968254,
 'gender_output_loss': 0.8461598141325845,
 'image_quality_output_acc': 0.5372023809523809,
 'image_quality_output_loss': 1.341307420579214,
 'loss': 9.695321953485882,
 'pose_output_acc': 0.7058531746031746,
 'pose_output_loss': 0.9762181753203982,
 'weight_output_acc': 0.6383928571428571,
 'weight_output_loss': 1.0732264357899863}

In [0]:
evaluate_model(model)

63/63 [==============================] - 3s 48ms/step


{'age_output_acc': 40.3274,
 'bag_output_acc': 58.9782,
 'emotion_output_acc': 70.6845,
 'footwear_output_acc': 62.5992,
 'gender_output_acc': 79.4147,
 'image_quality_output_acc': 53.7202,
 'pose_output_acc': 70.5853,
 'weight_output_acc': 63.8393}